# EOEPCA Data Access Validation and Usage Notebook

## Setup

In [14]:
import os
import requests
import json
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [15]:
load_eoepca_state()

In [16]:
platform_domain = os.environ.get("INGRESS_HOST")
data_access_domain = f'{os.environ.get("HTTP_SCHEME")}://eoapi.{platform_domain}'

print(f"Data Access API: {data_access_domain}")

Data Access API: https://eoapi.notebook-test.develop.eoepca.org


## Validate Data Access API Endpoints

In [17]:
endpoints = [
    ("STAC API Landing Page", f"{data_access_domain}/stac/"),
    ("STAC Swagger UI", f"{data_access_domain}/stac/api.html"),
    ("Raster API Swagger UI", f"{data_access_domain}/raster/api.html"),
    ("Vector API Swagger UI", f"{data_access_domain}/vector/api.html")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

STAC API Landing Page (https://eoapi.notebook-test.develop.eoepca.org/stac/): 200
STAC Swagger UI (https://eoapi.notebook-test.develop.eoepca.org/stac/api.html): 200
Raster API Swagger UI (https://eoapi.notebook-test.develop.eoepca.org/raster/api.html): 200
Vector API Swagger UI (https://eoapi.notebook-test.develop.eoepca.org/vector/api.html): 200


## STAC Collections and Items Inspection

In [18]:
collections_response = requests.get(f"{data_access_domain}/stac/collections")
collections = collections_response.json()
print("Collections:")
for collection in collections['collections']:
    print(f" - {collection['id']}")

Collections:
 - noaa-emergency-response
 - sentinel-2-iceland


## Load and Inspect Sample STAC Collection and Items

TODO: Why doesn't resource registration register it in?

In [19]:
collections_url = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/scripts/data-access/collections/sentinel-2-iceland/collections.json"
items_url = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/scripts/data-access/collections/sentinel-2-iceland/items.json"

collections_json = requests.get(collections_url).json()
items_json = requests.get(items_url).json()

## Ingestion of Sample Collection and Items

In [20]:
collections_endpoint = f"{data_access_domain}/stac/collections"
ingested_collection = requests.post(collections_endpoint, json=collections_json)
print(f"Collection ingestion status code: {ingested_collection.status_code}")
print(ingested_collection.json())

items_endpoint = f"{data_access_domain}/stac/collections/{collections_json['id']}/items"

successful_items = []
failed_items = []

# only take first 5 items for testing
items_json = items_json[:5]

for item in items_json:
    response = requests.post(items_endpoint, json=item)
    if response.status_code == 200:
        successful_items.append(item['id'])
    else:
        failed_items.append((item['id'], response.text))

print(f"Successfully ingested items: {successful_items}")
if failed_items:
    print(f"Failed items: {failed_items}")

Collection ingestion status code: 409
{'code': 'ConflictError', 'description': ''}
Successfully ingested items: []
Failed items: [('S2A_26WPS_20231109_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_27VWL_20231109_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_26WPS_20231108_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_26WNT_20231108_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_26WPT_20231108_0_L2A', '{"code":"ConflictError","description":""}')]


## STAC Items Search and Visualization (Post-Ingestion)

In [21]:
collection_check = requests.get(f"{collections_endpoint}/{collections_json['id']}")
print(f"Collection retrieval status: {collection_check.status_code}")

items_check = requests.get(items_endpoint)
print(f"Items retrieval status: {items_check.status_code}")

Collection retrieval status: 200
Items retrieval status: 200


In [22]:
search_url = f"{data_access_domain}/stac/search"
search_payload = {
    "bbox": [-25, 63, -24, 65],
    "datetime": "2023-01-01T00:00:00Z/2024-01-01T00:00:00Z",
    "limit": 5
}

response = requests.post(search_url, json=search_payload)
search_results = response.json()
print(json.dumps(search_results, indent=2)[:100])

{
  "type": "FeatureCollection",
  "links": [
    {
      "rel": "root",
      "type": "application/


In [23]:
if search_results['features']:
    stac_item = search_results['features'][0]
    visual_asset_url = stac_item['assets']['visual']['href']
    print(f"Visual asset URL: {visual_asset_url}")

    # image_response = requests.get(visual_asset_url)
    # img = Image.open(BytesIO(image_response.content))

    # plt.figure(figsize=(8, 8))
    # plt.imshow(img)
    # plt.axis('off')
    # plt.title(f"Raster Visualization: {stac_item['id']}")
    # plt.show()
else:
    print("No STAC items found.")

Visual asset URL: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/26/W/PS/2023/11/S2A_26WPS_20231109_0_L2A/TCI.tif


## Raster and Vector API Availability Checks

In [24]:
raster_health_url = f"{data_access_domain}/raster/healthz"
vector_health_url = f"{data_access_domain}/vector/healthz"

raster_health = requests.get(raster_health_url)
vector_health = requests.get(vector_health_url)

print(f"Raster API Health Check: {raster_health.status_code} - {raster_health.text}")
print(f"Vector API Health Check: {vector_health.status_code} - {vector_health.text}")


Raster API Health Check: 200 - {"database_online":true}
Vector API Health Check: 200 - {"ping":"pong!"}


## STAC Manager UI Check

In [25]:
stac_manager_ui_url = data_access_domain
response = requests.get(stac_manager_ui_url)
print(f"STAC Manager UI status code: {response.status_code}")
response.url

STAC Manager UI status code: 200


'https://eoapi.notebook-test.develop.eoepca.org/'